In [3]:
import pandas as pd

In [4]:
df= pd.read_csv("/content/train.csv")

In [5]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words("english")
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
def preprocess(text):
  if isinstance(text,str):
    text = re.sub(r"[^a-zA-Z\s]",'',text)
    text = re.sub(r"\d",'',text)
    text = re.sub(r"https\S+|http\S+|www\S+",'',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(i) for i in text if i not in set(stop)]
    return ' '.join(text)

  else:
    return ' '

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df["text"] = df["text"].apply(preprocess)

In [9]:
x = df["text"]
y = df["label"]
x

0        hous dem aid didnt even see comey letter jason...
1        ever get feel life circl roundabout rather hea...
2        truth might get fire octob tension intellig an...
3        video civilian kill singl us airstrik identifi...
4        print iranian woman sentenc six year prison ir...
                               ...                        
20795    rapper unload black celebr met donald trump el...
20796    green bay packer lost washington redskin week ...
20797    maci today grew union sever great name america...
20798    nato russia hold parallel exercis balkan press...
20799    david swanson author activist journalist radio...
Name: text, Length: 20800, dtype: object

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=3)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


In [13]:

tf = TfidfVectorizer(max_features=132680)
xtrain=tf.fit_transform(x_train).toarray()
xtest=tf.transform(x_test).toarray()

ytrain = y_train
ytest = y_test.values

In [14]:
import pickle
pickle.dump(tf,open("/content/Model.csv","wb"))

In [24]:
from sklearn.linear_model import LogisticRegression
nb = LogisticRegression()
nb.fit(xtrain,ytrain)

import joblib
joblib.dump(nb,"/content/NBModel.csv")
from sklearn.metrics import accuracy_score
ypred=nb.predict(xtest)
accuracy_score(ytest,ypred)

TypeError: ignored

In [16]:
dt = pd.read_csv("/content/test.csv")

In [17]:
dt.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [18]:
def preprocess(text):
  if isinstance(text,str):
    text = re.sub(r"[^a-zA-Z\s]",'',text)
    text = re.sub(r"\d",'',text)
    text = re.sub(r"https\S+|http\S+|www\S+",'',text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(i) for i in text if i not in set(stop)]
    return ' '.join(text)

  else:
    return ' '

In [19]:
dt["text"] = dt["text"].apply(preprocess)

In [28]:
import pickle

cv=pickle.load(open("/content/Model.csv","rb"))

In [29]:
x=dt["text"]
xt =cv.transform(x)

In [30]:
import joblib
l=joblib.load("/content/NBModel.csv")

In [31]:
dt["labels"] = l.predict(xt).tolist()

In [32]:
dt.head()

,id,title,author,text,labels
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,palo alto calif year scorn polit process silic...,0
1,20801,Russian warships ready to strike terrorists ne...,NaN,russian warship readi strike terrorist near al...,1
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,video nodapl nativ american leader vow stay wi...,1
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,first dont succeed tri differ sport tim tebow ...,0
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,min ago view comment like first time histori f...,1
